In [ ]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [1]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [4]:
l = 75
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,12);

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  0.368854 seconds (320.15 k allocations: 589.424 MB, 32.72% gc time)


In [6]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.107971 seconds (259.60 k allocations: 13.942 MB)
  0.080173 seconds (322.12 k allocations: 17.625 MB, 4.54% gc time)
  0.013778 seconds (7 allocations: 47.684 MB, 19.70% gc time)
  0.012109 seconds (7 allocations: 47.684 MB, 10.60% gc time)


In [7]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.338190 seconds (1.23 M allocations: 148.155 MB, 15.07% gc time)
  0.297563 seconds (1.21 M allocations: 147.300 MB, 15.78% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [8]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.046923 seconds (217.14 k allocations: 12.077 MB, 5.30% gc time)
  0.041683 seconds (211.89 k allocations: 11.764 MB)
  0.046537 seconds (217.14 k allocations: 12.077 MB, 5.47% gc time)
  0.018239 seconds (13 allocations: 47.684 MB, 7.72% gc time)
  0.015898 seconds (7 allocations: 47.684 MB, 4.35% gc time)
  0.049982 seconds (13 allocations: 47.684 MB, 1.66% gc time)


In [8]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

  3.900043 seconds (5.54 M allocations: 6.035 GB, 6.03% gc time)
  1.293228 seconds (176 allocations: 946.063 MB, 2.76% gc time)


In [1]:
using Tensors
using NullableArrays
using Iterators
using Base.size

include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end


genstensor (generic function with 2 methods)

In [2]:
T1 = randn(5,5);
r = randn(25, 25)
r1 = randn(25,10)
c = covbs(r, 5);
c1 = cov(r, corrected = false);



In [3]:
t = genstensor(Float64, 3,25)
bs = convert(BoxStructure{Float64}, t, 5);

In [4]:
function bcss3{T <: AbstractFloat}(bsdata::BoxStructure{T, 3}, m::Matrix{T})
    s = size(bsdata)
    s[3]  == size(m,2)||throw(DimensionMismatch("size of B1 $(s[3]) must equal to size of A $(size(m,2))"))
    m = slise(m, s[1])
    ret = NullableArray(Array{T, 3}, fill(size(m,1), 2)...)
    for i = 1:size(m,1)
      temp = NullableArray(Array{T, 3}, s[2], 1, 1)
      for k = 1:s[2]
          temp[k, 1, 1] = segmentmult1([i,k,1], bsdata, m)
      end
        for l = 1:j
            ret[i,j,l] = segmentmult1([j,1,1], temp[i,j], m, 1)
      end
   end
   ret
end


bcss3 (generic function with 1 method)

In [5]:
bcss3(bs, r1')

2x2 NullableArrays.NullableArray{Array{Float64,3},2}:
      5x5x5 Array{Float64,3}:
[:, :, 1] =
   5.74777  -10.6799    -10.9166     5.0843   13.0333  
   5.12134   -0.281218   -2.03015    5.12026   3.3496  
 -11.0883    23.7359     -3.60763    2.10315  17.8703  
 -21.9854     1.17153    -4.3479    -0.13072  -0.594912
   9.72706    3.98861     0.327007  -6.81443   0.67241 

[:, :, 2] =
  -4.55285     2.94061  2.33412  -14.2594    -2.4403 
   0.235536   -3.5433   2.46339   -6.39998   -1.94953
 -12.6079    -30.1052   6.96845    2.84582   18.8294 
   9.33275     8.25229  2.81774   -1.89907    3.89528
  13.1132     19.153    1.79056    6.18137  -16.0467 

[:, :, 3] =
  -0.0916293   -0.577611  -3.3156    12.2387        6.86757
  -8.647      -11.8704    -0.841567   0.000308356  -0.94929
 -26.3966     -29.3328     5.18613   10.0979       -9.48842
   2.60315     -0.478173  -2.54488   -8.35473      12.3497 
  -4.50449      3.03901   -3.40491    1.32979      -1.38473

[:, :, 4] =
  -3.03412   -1

In [4]:
bcss(c, r1) - bcss1(c, r1)

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0,

#NULL,
5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0,

5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0],5)

In [9]:
for f in (
    :+,
    :-,
)
    @eval begin
        @inline function $(f){S1, S2}(x::Nullable{Matrix{S1}}, y::Nullable{Matrix{S2}}, i::Int)
            Nullable($(f)(x.value, y.value), x.isnull | y.isnull)
        end
    end
end


In [3]:
r = randn(7,7)
a = covbs(r);


In [6]:
a*0

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [8]:
*(a, 0, true)


SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [9]:
a

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [7]:
j = 2
k = [1,2,3]
[j, k[2:end]...]

3-element Array{Int64,1}:
 2
 2
 3